In [ ]:
import base
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rushd as rd
import scipy as sp
import seaborn as sns

# enables concurrent editing of base.py
from importlib import reload
reload(base)

### Load data

Circuit tuning data (`data`)

In [ ]:
base_path = rd.datadir/'instruments'/'data'/'attune'/'kasey'

exp90_path = base_path/'2024.03.31_exp90'/'export'
exp90_2_path = base_path/'2024.04.02_exp90.2'/'export'
exp90_3_path = base_path/'2024.04.02_exp90.3'/'export'
exp90_4_path = base_path/'2024.04.05_exp90.4'/'export'
exp91_path = base_path/'2024.04.08_exp91'/'export'
exp92_path = base_path/'2024.04.12_exp92'/'export'
exp_elp60_path = rd.datadir/'instruments'/'data'/'attune'/'Emma'/'2024.06.09_EXP61'/'data_singlets'

plates = pd.DataFrame({
    'data_path': [exp90_path/'plate1', exp90_path/'plate2', 
                  exp90_2_path, exp90_4_path,
                  exp90_3_path/'plate1', exp90_3_path/'plate2', 
                  exp91_path/'plate1.1', exp91_path/'plate1.2', exp91_path/'plate1.3', 
                  exp91_path/'plate2.1', exp91_path/'plate2.2', exp91_path/'plate2.3',
                  exp92_path/'plate1.1', exp92_path/'plate1.2', exp92_path/'plate1.3', 
                  exp92_path/'plate2.1', exp92_path/'plate2.2', exp92_path/'plate2.3',
                  exp_elp60_path],
    
    'yaml_path': ([exp90_path/'exp90_plate1_wells.yaml', exp90_path/'exp90_plate2_wells.yaml', 
                   exp90_path/'exp90_plate2_wells.yaml', exp90_path/'exp90_plate1_wells.yaml',
                   exp90_path/'exp90_plate1_wells.yaml', exp90_path/'exp90_plate2_wells.yaml', ] +
                  [exp91_path/'exp91_plate1_wells.yaml']*3 + 
                  [exp91_path/'exp91_plate2.1_wells.yaml', exp91_path/'exp91_plate2.2_wells.yaml', exp91_path/'exp91_plate2.3_wells.yaml'] +
                  [exp92_path/'exp92_plate1_wells.yaml', exp92_path/'exp92_plate1.2_wells.yaml', exp92_path/'exp92_plate1_wells.yaml',
                   exp92_path/'exp92_plate2_wells.yaml', exp92_path/'exp92_plate2.2_wells.yaml', exp92_path/'exp92_plate2_wells.yaml',
                   exp_elp60_path/'metadata.yaml']
                  ),
    
    'biorep': ([1, 1, 
                2, 2, 
                3, 3,] + 
                [1, 2, 3,]*4 +
                [4]),
    
    'exp': (['exp90', 'exp90', 
             'exp90.2', 'exp90.4', 
             'exp90.3', 'exp90.3',] + 
            ['exp91']*6 + 
            ['exp92']*6 + 
            ['elp_exp61'])
})

output_path = rd.rootdir/'output'/'fig_model'
cache_path = rd.rootdir/'output'/'fig_overview'/'data.gzip'
metadata_path = rd.datadir/'projects'/'miR-iFFL'/'plasmids'/'construct-metadata.xlsx'

# Load data
data = pd.DataFrame()
if cache_path.is_file(): data = pd.read_parquet(cache_path)
else: 
    channel_list = ['mCherry-A','mRuby2-A','FSC-A','SSC-A','tagBFP-A','mGL-A']
    data = rd.flow.load_groups_with_metadata(plates, columns=channel_list)

    # Remove negative channel values
    for c in channel_list: data = data[data[c]>0]
    
    data.dropna(inplace=True)
    data.to_parquet(rd.outfile(cache_path))

# Add metadata for constructs
metadata = base.get_metadata(metadata_path, style='designs')
data = data.merge(metadata, how='left', on='construct')
display(data)

In [ ]:
# Add shades to teal color palette
#  light = EFS, miR-FF4, miRE-FF5, TSx2
#  dark = TSx4
metadata.loc[metadata['promoter']=='EFS', 'color'] = metadata.loc[metadata['promoter']=='EFS', 'color'].apply(base.get_dark_color)
metadata.loc[metadata['miR']=='miR.FF4', 'color'] = metadata.loc[metadata['miR']=='miR.FF4', 'color'].apply(base.get_dark_color)
metadata.loc[metadata['miR']=='miRE.FF5', 'color'] = metadata.loc[metadata['miR']=='miRE.FF5', 'color'].apply(base.get_light_color)
metadata.loc[((metadata['group']=='controller') & (metadata['ts_num']==2)), 'color'] = metadata.loc[((metadata['group']=='controller') & (metadata['ts_num']==2)), 'color'].apply(base.get_light_color)
metadata.loc[((metadata['group']=='controller') & (metadata['ts_num']==4)), 'color'] = metadata.loc[((metadata['group']=='controller') & (metadata['ts_num']==4)), 'color'].apply(base.get_dark_color)

# Create dicts to specify colors/markers
metadata_dict = metadata.set_index('construct').to_dict('dict')
main_palette = metadata_dict['color']
main_markers = metadata_dict['markers']

greys = matplotlib.colors.ListedColormap(matplotlib.colormaps['Greys'](np.linspace(0.2,0.7,256)))

In [ ]:
# Gate cells
gates = pd.DataFrame()
channel_list = ['mGL-A', 'mRuby2-A']
for channel in channel_list:
    gates[channel] = data[data['construct']=='UT'].groupby(['exp'])[channel].apply(lambda x: x.quantile(0.999))
gates.reset_index(inplace=True)

# Add missing gates
gates.loc[len(gates.index)] = ['exp90.4',0,0,]  
gates.loc[gates['exp']=='exp90.4', channel_list] = gates.loc[gates['exp']=='exp90.2', channel_list].values

# Indicate which channels are relevant for each experiment
gates.sort_values(['exp'], inplace=True)
gates['marker'] = 'mGL-A'
gates['output'] = 'mRuby2-A'

# Gate data by transfection marker expression
data = data.groupby('exp')[data.columns].apply(lambda x: base.gate_data(x,gates))
data.reset_index(inplace=True, drop=True)
df = data[(data['expressing']) & (data['construct']!='UT') & (data['exp']!='elp_exp61')]

In [ ]:
# Bin data and calculate statistics
df_quantiles, stats, _, fits = base.calculate_bins_stats(df, num_bins=10)
stats = stats.merge(metadata, how='left', on='construct')
fits = fits.merge(metadata, how='left', on='construct')

In [ ]:
ts_label = {'na': 'base', 'NT': 'OL', 'T': 'CL', 'none': '–'}
metadata['ts_label'] = metadata['ts_kind'].replace(ts_label)

Load modeling parameter sweeps

In [ ]:
simulation_path = rd.rootdir/'output'/'modeling'/'julia_param_sweeps'/'per_param'/'sweep_df.gzip'
sim_data = pd.DataFrame()
if simulation_path.is_file(): 
    sim_data = pd.read_parquet(simulation_path)

In [ ]:
# Normalize parameter values such that original (middle) value = 1
def normalize_param_val(df):
    d = df.copy()
    vals = d['param_val'].unique()
    d['param_val_norm'] = d['param_val'] / (sorted(vals)[int(len(vals)/2)])
    return d

sim_data = sim_data.groupby('param')[sim_data.columns].apply(normalize_param_val).reset_index(drop=True)
display(sim_data)

In [ ]:
display(sim_data['param'].unique())

In [ ]:
# Compute value of unregulate gene
alpha_rna = 4.67e-2     # params from `miR_iFFL.jl`
delta_mrna = 2.88e-4
alpha_p = 3.33e-4
delta_p = 9.67e-5

sim_data['unreg'] = sim_data['copy_num'] * (alpha_rna * alpha_p) / (delta_mrna * delta_p)

In [ ]:
# Compute instantaneous slope for each param & param_val 
def get_slope_instant(df, x, y):
    d = df.sort_values(x)
    return (list(d[y])[-1] - list(d[y])[-2]) / (list(d[x])[-1] - list(d[x])[-2])

col_list = ['copy_num','protein']
slopes = sim_data.groupby(['param','param_val','param_val_norm'])[sim_data.columns].apply(lambda x: get_slope_instant(x, *col_list)).rename('slope').reset_index()

alpha_rna = 4.67e-2     # params from `miR_iFFL.jl`
delta_mrna = 2.88e-4
alpha_p = 3.33e-4
delta_p = 9.67e-5
slopes['base_norm_factor'] = (delta_mrna * delta_p) / (alpha_rna * alpha_p)

def modify_norm_factor(df):
    param = df['param'].values[0]
    d = df.copy()
    if param in ['α_im', 'α_p']: 
        d['norm_factor'] = d['base_norm_factor'] * d['param_val_norm'].values[0]
    elif param in ['δ_m', 'δ_p']: 
        d['norm_factor'] = d['base_norm_factor'] / d['param_val_norm'].values[0]
    else:
        d['norm_factor'] = d['base_norm_factor']
    return d['norm_factor']

result = slopes.groupby(['param','param_val_norm'])[slopes.columns].apply(modify_norm_factor).rename('norm_factor').reset_index()
slopes['norm_factor'] = result['norm_factor']
slopes['slope_norm'] = slopes['slope'] * slopes['norm_factor']
display(slopes)

### Set up figure

In [ ]:
base_size = base.font_sizes['base_size']
smaller_size = base.font_sizes['smaller_size']

sns.set_style('ticks')
sns.set_context('paper', font_scale=1.0, rc={'font.size': base_size, 'font.family': 'sans-serif', 'font.sans-serif':['Arial']})
plt.rcParams.update({'axes.titlesize': base_size, 'axes.labelsize': base_size, 'xtick.labelsize': smaller_size, 'ytick.labelsize': smaller_size,
                     'pdf.fonttype': 42, 
                     'ytick.major.size': 3, 'xtick.major.size': 3, 'ytick.minor.size': 2, 'ytick.major.pad': 2, 'xtick.major.pad': 2, 
                     'lines.linewidth': 1,
                     'axes.spines.right': False, 'axes.spines.top': False, 'axes.labelpad': 2})

In [ ]:
# Create the overall figure, gridspec, and add subfigure labels
first_col = 2.75
fig = plt.figure(figsize=(6.8504,7.875))
fig_gridspec = matplotlib.gridspec.GridSpec(5, 3, figure=fig,
    wspace=0.4, hspace=0.4, height_ratios=[1.5+1/12]*3+[1.625,1.5], width_ratios=[first_col,(6.8504-first_col)/2,(6.8504-first_col)/2])
subfigures = {
    'A': fig.add_subfigure(fig_gridspec[:3,0]),
    'B': fig.add_subfigure(fig_gridspec[0,1:]),
    'C': fig.add_subfigure(fig_gridspec[1,1:]),
    'D': fig.add_subfigure(fig_gridspec[2,1:]),
    'E': fig.add_subfigure(fig_gridspec[3,0]),
    '': fig.add_subfigure(fig_gridspec[3,1:]),
    'G': fig.add_subfigure(fig_gridspec[4,0]),
    'H': fig.add_subfigure(fig_gridspec[4,1:]),
    #'': fig.add_subfigure(fig_gridspec[4,2]),
}
for label, subfig in subfigures.items():
    subfig.add_artist(matplotlib.text.Text(x=0, y=1, text=f'{label}', fontsize=base.font_sizes['subpanel_label'], 
                                           fontweight='bold', verticalalignment='top',transform=subfig.transSubfigure))
scatter_kwargs = dict(s=4, jitter=0.1, linewidth=0.5, edgecolor='white')
xlim = (2e2,6e4)
ylim = (2e1,1e5)

fig_name = 'fig_model.pdf'
fig_path = rd.datadir/'manuscripts'/'2024_miR-iFFL'/'figures'/'links'/fig_name
fig.savefig(rd.outfile(output_path/fig_name))
fig.savefig(rd.outfile(fig_path))

In [ ]:
subfig = subfigures['B']
rd.plot.adjust_subplot_margins_inches(subfig, left=1.4, bottom=0.45, top=0.35, right=0.3)
axes = subfig.subplots(1,2, gridspec_kw=dict(wspace=0.5))

# model: basic params
ax = axes[0]
yticks = np.linspace(0,1e4,5)
plot_df = sim_data[sim_data['param']=='Rtot']
sns.lineplot(data=plot_df[plot_df['param_val_norm']==1], x='copy_num', y='protein', color='black', ax=ax, linewidth=1.5,
             errorbar=None,)
sns.lineplot(data=plot_df[plot_df['param_val_norm']==1], x='copy_num', y='unreg', color=base.colors['gray'], ax=ax,
             errorbar=None,)
ax.set(xticks=np.linspace(0,100,5), yticks=yticks, xlabel='copy #', ylim=(0,12e3),
       yticklabels=[f'{float(l)/1000:.1f}k' for l in yticks], title='Initial parameters', ylabel='protein (#)')
ax.axvspan(55,100, color=base.get_light_color(base.colors['gray']), alpha=0.2)
setpoint = plot_df.loc[(plot_df['param_val_norm']==1) & (plot_df['copy_num']==50), 'protein'].values[0]
ax.axhline(setpoint, color=base.colors['teal'], ls=':', xmax=100, zorder=1)

# model: Rtot sweep
ax = axes[1]
sns.lineplot(data=plot_df, x='copy_num', y='protein', hue='param_val_norm', palette=greys, 
             ax=ax, hue_norm=matplotlib.colors.LogNorm(), legend=False)
sns.lineplot(data=plot_df[plot_df['param_val_norm']==1], x='copy_num', y='protein', color='black', ax=ax, linewidth=1.5,
             errorbar=None,)
ax.set(xticks=np.linspace(0,100,5), yticks=yticks, xlabel='copy #', ylim=(0,12e3),
       #yticklabels=[f'{float(l)/1000:.1f}k' for l in yticks],)
       yticklabels=['']*len(yticks), ylabel='')
ax.set_title('Total RISC, $R_{tot}$', color=base.colors['green'])

fig.savefig(rd.outfile(output_path/fig_name))
fig.savefig(rd.outfile(fig_path))

In [ ]:
subfig = subfigures['C']
rd.plot.adjust_subplot_margins_inches(subfig, left=1.4, bottom=0.45, top=0.35, right=0.3)
axes = subfig.subplots(1,2, gridspec_kw=dict(wspace=0.5))

# model: k_mRNA_bind vs protein at set copy #
ax = axes[0]
plot_df = slopes[(slopes['param']=='α_im')]
sns.scatterplot(data=plot_df, x='param_val_norm', y='slope_norm', hue='param_val_norm', palette=greys, 
             ax=ax, hue_norm=matplotlib.colors.LogNorm(), legend=False, s=2.25**2*3.14, linewidth=0.5, edgecolor='white')
sns.scatterplot(data=plot_df[plot_df['param_val_norm']==1], x='param_val_norm', y='slope_norm', color='black', 
             ax=ax, hue_norm=matplotlib.colors.LogNorm(), legend=False, s=2.25**2*3.14, linewidth=0.5, edgecolor='white')
ax.set(xlabel='Norm. $α_{RNA}$', ylabel='slope')
ax.set_title('Transcription, $α_{RNA}$', color=base.colors['blue'])
threshold = plot_df['param_val_norm'].unique()[2] + (plot_df['param_val_norm'].unique()[3] - plot_df['param_val_norm'].unique()[2])/2
ax.axvspan(threshold,ax.get_xlim()[1], color=base.get_light_color(base.colors['gray']), alpha=0.2)

# data: miRE-FF4, EF1a vs EFS
ax = axes[1]
promoter_order = ['EFS','EF1a']
plot_df2 = fits[(((fits['miR']=='miRE.FF4') & (fits['group']=='controller') & (fits['design']==1) & (fits['ts'].isin(['FF4x1','FF6x1']))) &
                 fits['promoter'].isin(promoter_order))]
plot_df2.sort_values(['group','ts_kind'], inplace=True)
plot_df2.sort_values('promoter', ascending=False, inplace=True)

# shift xticks to add more space between promoter groups
buffer = 0.6
xlim_adjusted = (-0.7, len(plot_df2['construct'].unique())-0.3+buffer*(len(promoter_order)-1))
xtick_locs = np.concatenate([[x+i*(2+buffer) for x in range(2)] for i in range(len(promoter_order))])
construct_loc = {k:v for k,v in zip(plot_df2['construct'].unique(), xtick_locs)}
plot_df2['construct_loc'] = plot_df2['construct'].replace(construct_loc)
metadata['construct_loc'] = metadata['construct'].map(construct_loc)
m = metadata.dropna()
m['construct_loc'] = m['construct_loc'].astype(str)

for construct, group in plot_df2.groupby('construct', sort=False):
    sns.stripplot(data=group, x='construct_loc', y='slope', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], **scatter_kwargs, native_scale=True)
ax.set(title='Experimental data', xlabel='', ylim=(0,1.2), ylabel='slope', xticks=xtick_locs, xlim=xlim_adjusted,
       yticks=[0,0.25,0.5,0.75,1], yticklabels=['0.0','','0.5','','1.0'], xticklabels=['']*len(ax.get_xticks()))
marker_baseline = fits.loc[(fits['group']=='marker'), 'slope'].mean()
ax.axhline(marker_baseline, color='black', ls=':')
# add promoter labels
for j, prom in enumerate(promoter_order):
       yloc = -6
       ax.annotate(prom, (0.5+j*(2+buffer),yloc), xycoords=('data','axes points'), ha='center', va='top', fontsize=smaller_size)

fig.savefig(rd.outfile(output_path/fig_name))
fig.savefig(rd.outfile(fig_path))

In [ ]:
subfig = subfigures['D']
rd.plot.adjust_subplot_margins_inches(subfig, left=1.4, bottom=0.45, top=0.35, right=0.3)
axes = subfig.subplots(1,2, gridspec_kw=dict(wspace=0.5))

# model: r_drosha sweep
ax = axes[0]
yticks = np.linspace(0,1e4,5)
plot_df = sim_data[sim_data['param']=='r_drosha']
sns.lineplot(data=plot_df, x='copy_num', y='protein', hue='param_val_norm', palette=greys, 
             ax=ax, hue_norm=matplotlib.colors.LogNorm(), legend=False)
sns.lineplot(data=plot_df[plot_df['param_val_norm']==1], x='copy_num', y='protein', color='black', ax=ax, linewidth=1.5,
             errorbar=None,)

ax.set(xticks=np.linspace(0,100,5), yticks=yticks, xlabel='copy #', ylim=(0,12e3),
       yticklabels=[f'{float(l)/1000:.1f}k' for l in yticks], ylabel='protein (#)')
ax.set_title('Drosha processing, $r_{drosha}$', color=base.colors['purple'])

# data: miRE-FF4 vs miR-FF4
ax = axes[1]
biorep = 1
plot_df = df_quantiles[((df_quantiles['miR'].isin(['miRE.FF4','miR.FF4'])) & (df_quantiles['group']=='controller') & (df_quantiles['design']==1) & (df_quantiles['ts'].isin(['FF4x1','FF5x1','FF6x1']))) &
                       (df_quantiles['promoter']=='EF1a') & (df_quantiles['biorep']==biorep)]
display(plot_df)

sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=main_palette, 
             legend=False, dashes=False, style='construct', markers=main_markers, ax=ax, markersize=4, markeredgewidth=0.5,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlabel='marker', xlim=xlim, ylim=ylim, yticks=(1e2,1e3,1e4,1e5), title='Experimental data')
sns.despine(ax=ax)
ax.minorticks_off()

fig.savefig(rd.outfile(output_path/fig_name))
fig.savefig(rd.outfile(fig_path))

In [ ]:
subfig = subfigures['E']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.6, bottom=0.45, top=0.4, right=0.35)
axes = subfig.subplots(1,2, gridspec_kw=dict(wspace=0.2, width_ratios=(1,0.75)))

# model: k_mRNA_bind sweep
ax = axes[0]
yticks = np.linspace(0,1e4,5)
plot_df = sim_data[sim_data['param']=='k_mRNA_bind']
sns.lineplot(data=plot_df, x='copy_num', y='protein', hue='param_val', palette=greys, 
             ax=ax, hue_norm=matplotlib.colors.LogNorm(), legend=False)
sns.lineplot(data=plot_df[plot_df['param_val_norm']==1], x='copy_num', y='protein', color='black', ax=ax, linewidth=1.5,
             errorbar=None,)
ax.set(xticks=np.linspace(0,100,5), yticks=yticks, xlabel='copy #', ylim=(0,12e3),
       yticklabels=[f'{float(l)/1000:.1f}k' for l in yticks], ylabel='protein (#)')
ax.set_title('RISC-mRNA binding,\n$k_{mRNA,bind}$', color=base.colors['red'])

# add annotations
param_val_subset = [p for i,p in enumerate(plot_df['param_val_norm'].unique()) if i%2==1]
plot_df = sim_data[(sim_data['param']=='k_mRNA_bind') & (sim_data['param_val_norm'].isin(param_val_subset)) & (plot_df['copy_num'].isin([25,75]))]
reds = sns.light_palette(color=base.colors['red'], n_colors=7)[1:6]
greys2 = greys.colors
greys2[128] = (0,0,0,1)
greys_subset = [l.tolist() for i,l in enumerate(greys2) if i%64==0]
greys_subset.append(greys2[-1])
ax.axvline(x=25, color=reds[1], ymax=(11e3)/(12e3))
ax.axvline(x=75, color=reds[4], ymax=(11e3)/(12e3))
sns.scatterplot(data=plot_df, x='copy_num', y='protein', hue='param_val', palette=greys_subset,
                ax=ax, s=2.25**2*3.14, linewidth=0.5, edgecolor='white', legend=False, zorder=100)

# model: k_mRNA_bind vs protein at set copy #
ax = axes[1]
plot_df = sim_data[(sim_data['param']=='k_mRNA_bind') & (sim_data['copy_num'].isin([25,75]))].copy()
sns.lineplot(data=plot_df, x='param_val_norm', y='protein', hue='copy_num', palette={25: reds[1], 75: reds[4]}, ax=ax, 
             legend='full', hue_norm=matplotlib.colors.Normalize(vmin=0, vmax=100,))
plot_df.sort_values('copy_num', inplace=True)
sns.scatterplot(data=plot_df[(plot_df['param_val_norm'].isin(param_val_subset))], x='param_val_norm', y='protein', hue='param_val_norm', palette=greys_subset,
                ax=ax, s=2.25**2*3.14, linewidth=0.5, edgecolor='white', legend=False, zorder=100)
ax.set(yticks=yticks, xlabel='Norm. $k_{mRNA,bind}$', ylim=(0,12e3),
       yticklabels=['']*len(yticks), ylabel='')
sns.move_legend(ax, labels=[25,75], loc='upper right', bbox_to_anchor=(1,1), frameon=False,
                fontsize=smaller_size, handlelength=1, labelspacing=0.3, title='copy #',
                title_fontsize=smaller_size)

fig.savefig(rd.outfile(output_path/fig_name))
fig.savefig(rd.outfile(fig_path))

In [ ]:
subfig = subfigures['']
rd.plot.adjust_subplot_margins_inches(subfig, left=1.4, bottom=0.45, top=0.4, right=0.3)
axes = subfig.subplots(1,2, gridspec_kw=dict(width_ratios=(1,1), wspace=0.5))

# data: miRE-FF4 vs miRE-FF5 stats
ax = axes[0]
miR_order = ['miRE.FF4','miRE.FF5']
plot_df = stats[((stats['group']=='controller') & (stats['design']==1) & (stats['miR'].isin(miR_order)) & (stats['ts'].isin(['FF4x1','FF5x1','FF6x1']))) &
                (stats['promoter']=='EF1a')].copy()
plot_df['miR'] = plot_df['miR'].astype(pd.api.types.CategoricalDtype(categories=miR_order, ordered=True))
xlim = (-0.5, len(plot_df['miR'].unique())-0.5)

for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='miR', y='output_gmean', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], **scatter_kwargs)
ax.set(title='Mean output', xlim=xlim, xlabel='', ylabel='', yscale='log',)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_off()
ax.set_xticklabels([l.get_text().replace('.','-\n') for l in ax.get_xticklabels()])
sns.despine(ax=ax)

# data: vary ts num
ax = axes[1]
plot_df = stats[(stats['group']=='controller') & (stats['miR']=='miRE.FF4') & (stats['design']==1) & (stats['promoter']=='EF1a')].copy()
xlim = (-0.5, len(plot_df['ts_num'].unique())-0.5)

for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts_num', y='output_gmean', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], **scatter_kwargs)
ax.set(title='Mean output', xlim=xlim, xlabel='# target sites', ylabel='', yscale='log',)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_off()
sns.despine(ax=ax)

fig.savefig(rd.outfile(output_path/fig_name))
fig.savefig(rd.outfile(fig_path))

In [ ]:
subfig = subfigures['G']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.6, bottom=0.3, top=0.4, right=1.22)
axes = subfig.subplots(1,1)

# model: alpha_p sweep
ax = axes
yticks = np.linspace(0,1e4,5)
plot_df = sim_data[sim_data['param']=='α_p']
sns.lineplot(data=plot_df, x='copy_num', y='protein', hue='param_val', palette=greys, 
             ax=ax, hue_norm=matplotlib.colors.LogNorm(), legend=False)
sns.lineplot(data=plot_df[plot_df['param_val_norm']==1], x='copy_num', y='protein', color='black', ax=ax, linewidth=1.5,
             errorbar=None,)
ax.set(xticks=np.linspace(0,100,5), yticks=yticks, xlabel='copy #', ylim=(0,12e3),
       yticklabels=[f'{float(l)/1000:.1f}k' for l in yticks], ylabel='protein (#)')
ax.set_title('Free mRNA\ntranslation, $α_{p}$', color=base.colors['orange'])

fig.savefig(rd.outfile(output_path/fig_name))
fig.savefig(rd.outfile(fig_path))

In [ ]:
subfig = subfigures['H']
rd.plot.adjust_subplot_margins_inches(subfig, left=1.4, bottom=0.3, top=0.4, right=1.74)
axes = subfig.subplots(1,1,)

# model: k_miRNA_deg sweep
ax = axes
yticks = np.linspace(0,1e4,5)
plot_df = sim_data[sim_data['param']=='k_miRNA_deg']
sns.lineplot(data=plot_df, x='copy_num', y='protein', hue='param_val', palette=greys, 
             ax=ax, hue_norm=matplotlib.colors.LogNorm(), legend=False)#legend='full')
sns.lineplot(data=plot_df[plot_df['param_val_norm']==1], x='copy_num', y='protein', color='black', ax=ax, linewidth=1.5,
             errorbar=None,)
ax.set(xticks=np.linspace(0,100,5), yticks=yticks, xlabel='copy #', ylim=(0,6e3),
       title='Bound microRNA\ndegradation, $k_{miRNA,deg}$',
       yticklabels=[f'{float(l)/1000:.1f}k' for l in yticks], ylabel='protein (#)')

fig.savefig(rd.outfile(output_path/fig_name))
fig.savefig(rd.outfile(fig_path))